
## 1. **Exploración y Comprensión de la Estructura de Datos**

### a. Categorías y Atributos
- **Obtener el árbol de categorías:**  
  Utilizaremos el endpoint `/sites/MLA/categories` para recuperar todas las categorías del sitio (por ejemplo, Mercado Libre Argentina). Esto nos ayudará a entender en qué contextos se publican los ítems y a segmentarlos según su naturaleza.
  
- **Detalle de una categoría:**  
  Con endpoints como `/categories/{id}` (por ejemplo, `/categories/MLA1744` para “Autos y Camionetas”), podemos conocer la estructura jerárquica, los atributos obligatorios y opcionales, y la configuración específica de cada categoría. Esto es clave para saber qué campos (títulos, descripciones, imágenes, atributos técnicos, etc.) tenemos disponibles para comparar los productos.

- **Atributos específicos y valores top:**  
  Usaremos también `/categories/{id}/attributes` para conocer los atributos específicos de una categoría. Además, el endpoint de **top values** nos ayudará a identificar cuáles son los valores más comunes para ciertos atributos (por ejemplo, las marcas más usadas), lo que puede servir para validar o enriquecer nuestro análisis.

### b. Predictor de Categorías
- Con el recurso `/domain_discovery/search` podremos, a partir de un título de producto, predecir la categoría a la que pertenece. Esto es útil si queremos validar que estamos comparando ítems dentro de la misma categoría o si necesitamos filtrar productos que efectivamente son similares.

---

## 2. **Extracción de Datos: Cómo Recopilar la Información**

### a. Extracción Masiva mediante Búsqueda
- **Uso del endpoint de búsqueda:**  
  Emplearemos la llamada a `/sites/MLA/search` con parámetros `category`, `q`, `limit` y `offset` para extraer un conjunto representativo de publicaciones. Por ejemplo:
  ```bash
  curl -X GET "https://api.mercadolibre.com/sites/MLA/search?category=MLA1000&offset=0&limit=50"
  ```
  Esto nos permitirá recopilar títulos, imágenes y otros datos relevantes de cada ítem.

### b. Paginación y Manejo de Resultados
- **Limit y Offset:**  
  La documentación explica cómo utilizar los parámetros `limit` y `offset` para obtener bloques de resultados. Esto es esencial para recolectar una cantidad suficiente de datos sin sobrecargar el sistema y para realizar un muestreo representativo de los productos.

---



### Importación de librerías

In [2]:
import requests
import json

## Obtener las Categorías del Sitio

### Mercado Libre Colombia

In [4]:


# Definimos la URL del endpoint de categorías para MLA
url_categorias = "https://api.mercadolibre.com/sites/MCO/categories"

# Realizamos la petición GET
respuesta = requests.get(url_categorias)

# Verificamos que la petición se haya realizado con éxito
if respuesta.status_code == 200:
    categorias = respuesta.json()
    # Imprimimos las primeras categorías para ver el formato
    print("Número de categorías obtenidas:", len(categorias))
    print(json.dumps(categorias, indent=4, ensure_ascii=False))
else:
    print("Error en la solicitud:", respuesta.status_code)


Número de categorías obtenidas: 32
[
    {
        "id": "MCO1747",
        "name": "Accesorios para Vehículos"
    },
    {
        "id": "MCO441917",
        "name": "Agro"
    },
    {
        "id": "MCO1403",
        "name": "Alimentos y Bebidas"
    },
    {
        "id": "MCO1071",
        "name": "Animales y Mascotas"
    },
    {
        "id": "MCO1367",
        "name": "Antigüedades y Colecciones"
    },
    {
        "id": "MCO1368",
        "name": "Arte, Papelería y Mercería"
    },
    {
        "id": "MCO1384",
        "name": "Bebés"
    },
    {
        "id": "MCO1246",
        "name": "Belleza y Cuidado Personal"
    },
    {
        "id": "MCO40433",
        "name": "Boletas para Espectáculos"
    },
    {
        "id": "MCO1039",
        "name": "Cámaras y Accesorios"
    },
    {
        "id": "MCO1743",
        "name": "Carros, Motos y Otros"
    },
    {
        "id": "MCO1051",
        "name": "Celulares y Teléfonos"
    },
    {
        "id": "MCO1648",
        

## Explorar el Detalle de una Categoría Específica

###
 {
        "id": "MCO1000",
        "name": "Electrónica, Audio y Video"
 }

In [5]:
# Definimos el ID de la categoría que queremos explorar
categoria_id = "MCO1000"
url_detalle_categoria = f"https://api.mercadolibre.com/categories/{categoria_id}"

respuesta_categoria = requests.get(url_detalle_categoria)

if respuesta_categoria.status_code == 200:
    detalle_categoria = respuesta_categoria.json()
    # Mostramos información relevante: nombre, atributos obligatorios, estructura jerárquica, etc.
    print("Detalle de la categoría:")
    print(json.dumps(detalle_categoria, indent=4, ensure_ascii=False))
else:
    print("Error en la solicitud de detalle de categoría:", respuesta_categoria.status_code)


Detalle de la categoría:
{
    "id": "MCO1000",
    "name": "Electrónica, Audio y Video",
    "picture": "https://http2.mlstatic.com/storage/categories-api/images/943ec641-717e-49cb-8a34-2f40ba367f5a.png",
    "permalink": "https://www.mercadolibre.com.co/c/electronica-audio-y-video",
    "total_items_in_this_category": 1118751,
    "path_from_root": [
        {
            "id": "MCO1000",
            "name": "Electrónica, Audio y Video"
        }
    ],
    "children_categories": [
        {
            "id": "MCO3690",
            "name": "Accesorios Audio y Video",
            "total_items_in_this_category": 69026
        },
        {
            "id": "MCO431414",
            "name": "Accesorios para TV",
            "total_items_in_this_category": 38198
        },
        {
            "id": "MCO3835",
            "name": "Audio",
            "total_items_in_this_category": 426937
        },
        {
            "id": "MCO5054",
            "name": "Cables",
            "total_i

# Realizar una Búsqueda de Ítems en una Categoría

"Electrónica, Audio y Video"

In [8]:
import requests
import pandas as pd
import time
from tqdm import tqdm

# Definir la categoría a analizar (Ej: Electrónica, Audio y Video en Colombia)
CATEGORY_ID = "MCO1000"  # Cambia este ID para otra categoría si es necesario

# Definir la cantidad de productos a extraer (máximo 1000 por consulta)
MAX_PRODUCTS = 1000  # Ajustable según necesidad
OFFSET_STEP = 50   # Mercado Libre devuelve datos en bloques de 50

# Lista para almacenar los productos extraídos
products = []

# URL base de la API de Mercado Libre
API_URL = f"https://api.mercadolibre.com/sites/MCO/search?category={CATEGORY_ID}&offset="

# Bucle para obtener los datos paginados
for offset in tqdm(range(0, MAX_PRODUCTS, OFFSET_STEP), desc="Extrayendo productos"):
    url = f"{API_URL}{offset}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        # Se añaden todos los campos del producto (cada 'item' es un diccionario con toda la información)
        for item in data["results"]:
            products.append(item)
    else:
        print(f"Error en la consulta de offset {offset}: {response.status_code}")
    
    time.sleep(0.5)  # Esperar para no saturar la API

# Convertir los datos a un DataFrame de Pandas utilizando json_normalize para aplanar estructuras anidadas
df = pd.json_normalize(products)

# Mostrar las primeras filas del DataFrame (head)
df.head()


Extrayendo productos: 100%|██████████| 20/20 [00:25<00:00,  1.26s/it]


,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,sanitized_title,permalink,buying_mode,site_id,...,variations_data.178833139685.attributes,variations_data.178833139685.attribute_combinations,variations_data.180242557470.thumbnail,variations_data.180242557470.ratio,variations_data.180242557470.name,variations_data.180242557470.pictures_qty,variations_data.180242557470.price,variations_data.180242557470.user_product_id,variations_data.180242557470.attributes,variations_data.180242557470.attribute_combinations
0,MCO2739654108,Audífonos Inalámbricos Bluetooth Sleve Pulse A...,new,693995-MLU73095898992_112023,MCO28584565,gold_special,,https://www.mercadolibre.com.co/audifonos-inal...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCO1432939685,Smart Tv Samsung Un65du8200kxzl 65 4k Uhd,new,991818-MLU76534344606_052024,MCO37334512,gold_pro,,https://www.mercadolibre.com.co/smart-tv-samsu...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCO1412662267,Parlante Bose S1pro+ Plus Con Batería Color Negro,new,866492-MLU70777686720_072023,MCO44727560,gold_special,,https://www.mercadolibre.com.co/parlante-bose-...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCO1477689665,Smart Tv Samsung Crystal Un43du7000kxzl 43'' 4...,new,754085-MLU77105577040_062024,MCO37925187,gold_special,,https://www.mercadolibre.com.co/smart-tv-samsu...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCO1450614137,Audifonos Jbl Tune 520 Bt Bluetooth On Ear Col...,new,618264-MLU76488383390_052024,MCO37266173,gold_special,,https://www.mercadolibre.com.co/audifonos-jbl-...,buy_it_now,MCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Elección de una Categoría Específica para el Challenge

## Introducción

Para abordar el challenge de identificar y agrupar productos similares en el marketplace, se ha decidido centrar el análisis en una única categoría. Esta estrategia permite desarrollar un prototipo robusto y profundo, además de facilitar la validación de los resultados.

## Motivación y Justificación de la Elección

La decisión de enfocarse en la categoría **"Electrónica, Audio y Video" (ID: MCO1000)** se fundamenta en diversas razones, tanto metodológicas como personales, desde la perspectiva del participante:

- **Afinidad por la Tecnología:**  
  El candidato muestra un especial interés en el ámbito tecnológico, y la categoría de Electrónica, Audio y Video abarca una amplia gama de productos innovadores. Esto resulta motivador para aplicar técnicas avanzadas de procesamiento de lenguaje natural y análisis de imágenes. Así mismo, por una experiencia personal al usar la aplicación de Mercado Libre para poner en venta unos transistores y sensores de temperatura. 

- **Calidad y Abundancia de Datos:**  
  Esta categoría ofrece una gran cantidad de publicaciones con información detallada (títulos, descripciones, imágenes, etc.), lo que facilita la extracción y el preprocesamiento de datos, permitiendo la construcción de modelos más precisos para comparar y agrupar ítems.

- **Homogeneidad en los Atributos:**  
  Trabajar en una sola categoría simplifica el análisis, ya que los productos comparten características y atributos similares. Esto reduce la complejidad en el preprocesamiento y mejora la consistencia en el análisis de similitud.

- **Enfoque y Profundidad del Análisis:**  
  Centrarse en una única categoría permite profundizar en el estudio de sus características particulares y optimizar los algoritmos de similitud y clustering, evitando la dispersión que implicaría abordar múltiples dominios simultáneamente.
